In [17]:
import os
import cv2
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [18]:
DATA_DIR = r"archive (1)/processed_data"

In [19]:
def createdataframe(dir_path):
    image_paths = []
    labels = []

    for label in os.listdir(dir_path):
        label_path = os.path.join(dir_path, label)
        if os.path.isdir(label_path):
            for img in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, img))
                labels.append(label)

    return image_paths, labels


data = pd.DataFrame()
data['image'], data['label'] = createdataframe(DATA_DIR)

print(data.head())
print(data['label'].value_counts())

                                              image  label
0  archive (1)/processed_data\angry\angry_00000.jpg  angry
1  archive (1)/processed_data\angry\angry_00001.jpg  angry
2  archive (1)/processed_data\angry\angry_00002.jpg  angry
3  archive (1)/processed_data\angry\angry_00003.jpg  angry
4  archive (1)/processed_data\angry\angry_00004.jpg  angry
label
happy       11398
neutral      8166
sad          6535
fear         5920
disgust      5920
angry        5920
surprise     5920
Name: count, dtype: int64


In [20]:
X = data['image']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [21]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_train.shape[1]
print("Classes:", le.classes_)

Classes: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [22]:
def extract_features(images):
    features = []

    for img_path in images:
        img = load_img(img_path, color_mode='grayscale', target_size=(48, 48))
        img = np.array(img)
        img = img.reshape(48, 48, 1)
        features.append(img)

    features = np.array(features)
    features = features / 255.0
    return features

In [23]:
X_train = extract_features(X_train)
X_test = extract_features(X_test)

print(X_train.shape, X_test.shape)

(39823, 48, 48, 1) (9956, 48, 48, 1)


In [24]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

c:\Users\imanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test)
)

Epoch 1/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 39s 59ms/step - accuracy: 0.3601 - loss: 1.6619 - val_accuracy: 0.4928 - val_loss: 1.3642
Epoch 2/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 33s 53ms/step - accuracy: 0.4727 - loss: 1.3985 - val_accuracy: 0.5360 - val_loss: 1.2576
Epoch 3/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.5004 - loss: 1.3225 - val_accuracy: 0.5468 - val_loss: 1.1783
Epoch 4/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.5178 - loss: 1.2697 - val_accuracy: 0.5592 - val_loss: 1.1359
Epoch 5/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step - accuracy: 0.5318 - loss: 1.2331 - val_accuracy: 0.5510 - val_loss: 1.1999
Epoch 6/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 32s 52ms/step - accuracy: 0.5466 - loss: 1.1993 - val_accuracy: 0.5699 - val_loss: 1.1368
Epoch 7/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 34s 54ms/step - accuracy: 0.5537 - loss: 1.1803 - val_accuracy: 0.5858 - val_loss: 1.0910
Epoch 8/20
623/623 ━━━━━━━━━━━━━━━━━━━━ 37s 60ms/step - accuracy: 0.5615 - loss: 1.1549 - 

In [26]:
model.save("emotion_model.h5")

In [27]:
model.save("emotion_model.keras")

In [28]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img

In [29]:
model = load_model("emotion_model.keras")
print("Model loaded successfully")

Model loaded successfully


c:\Users\imanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [30]:
labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [31]:
def predict_emotion(img_path):
    img = load_img(img_path, color_mode='grayscale', target_size=(48, 48))
    img = np.array(img)
    img = img.reshape(1, 48, 48, 1)
    img = img / 255.0

    prediction = model.predict(img)
    return labels[np.argmax(prediction)]

In [32]:
test_image = r"archive (1)/processed_data/happy/happy_01416.jpg"

In [33]:
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Predicted emotion: happy


In [34]:
import os
print(os.listdir(r"archive (1)/processed_data/happy")[:5])

['happy_01416.jpg', 'happy_01417.jpg', 'happy_01418.jpg', 'happy_01419.jpg', 'happy_01420.jpg']


In [35]:
test_image = r"archive (1)/processed_data/sad/sad_00012.jpg"
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicted emotion: sad


In [36]:
test_image = r"archive (1)/processed_data/angry/angry_00314.jpg"
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted emotion: angry


In [37]:
test_image = r"archive (1)/processed_data/disgust/disgust_00853.jpg"
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Predicted emotion: angry


In [38]:
test_image = r"archive (1)/processed_data/fear/fear_00258.jpg"
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted emotion: fear


In [39]:
test_image = r"archive (1)/processed_data/neutral/neutral_00544.jpg"
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted emotion: sad


In [40]:
test_image = r"archive (1)/processed_data/surprise/surprise_00399.jpg"
print("Predicted emotion:", predict_emotion(test_image))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Predicted emotion: surprise


In [41]:
import cv2

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

In [42]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.reshape(1, 48, 48, 1) / 255.0

        prediction = model.predict(face)
        emotion = labels[np.argmax(prediction)]

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, emotion, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Face Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.reshape(1, 48, 48, 1) / 255.0

        prediction = model.predict(face, verbose=0)
        emotion = labels[np.argmax(prediction)]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, emotion, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Face Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()